# co

> Covariance and Coherence Matrix Estimation

In [ ]:
#| default_exp co

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
# for testing
from cupy.testing import assert_array_almost_equal

In [ ]:
import numpy as np
import cupy as cp
import itertools
from decorrelation.shp import ks_test
import math
import zarr

In [ ]:
#| export
import cupy as cp
from typing import Union

## Covariance and Coherence Matrix Estimator

In [ ]:
#| export
_emperical_co_kernel = cp.ElementwiseKernel(
    'raw T rslc, raw bool is_shp, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win',
    'raw T cov, raw T coh',
    '''
    if (i >= nlines*width) return;
    int az_win = 2*az_half_win+1;
    int r_win = 2*r_half_win+1;
    int win = az_win*r_win;
    
    int ref_az = i/width;
    int ref_r = i -ref_az*width;

    int sec_az, sec_r;

    int m,j; // index of each coherence matrix
    int k,l; // index of search window
    T _cov; // covariance
    float _amp2_m; // sum of amplitude square for image i
    float _amp2_j; // sum of amplitude aquare for image j
    int rslc_inx_m, rslc_inx_j;
    int n; // number of shp

    for (m = 0; m < nimages; m++) {
        for (j = 0; j < nimages; j++) {
            _cov = T(0.0, 0.0);
            _amp2_m = 0.0;
            _amp2_j = 0.0;
            n = 0;
            for (k = 0; k < az_win; k++) {
                for (l = 0; l < r_win; l++) {
                    sec_az = ref_az-az_half_win+k;
                    sec_r = ref_r-r_half_win+l;
                    if (is_shp[i*win+k*r_win+l] && sec_az >= 0 && sec_az < nlines && sec_r >= 0 && sec_r < width) {
                        rslc_inx_m = (sec_az*width+sec_r)*nimages+m;
                        rslc_inx_j = (sec_az*width+sec_r)*nimages+j;
                        _amp2_m += norm(rslc[rslc_inx_m]);
                        _amp2_j += norm(rslc[rslc_inx_j]);
                        _cov += rslc[rslc_inx_m]*conj(rslc[rslc_inx_j]);
                        n += 1;
                        //if (i == 0 && m ==3 && j == 1) {
                        //    printf("%f",_cov.real());
                        //}
                    }
                }
            }
            cov[(i*nimages+m)*nimages+j] = _cov/(float)n;
            //if ( i == 0 && m==3 && j ==1 ) printf("%d",((i*nimages+m)*nimages+j));
            _amp2_m = sqrt(_amp2_m*_amp2_j);
            coh[(i*nimages+m)*nimages+j] = _cov/_amp2_m;
        }
    }
    ''',
    name = 'emperical_co_kernel',reduce_dims = False,no_return=True
)

In [ ]:
#| export
def emperical_co(rslc:cp.ndarray, # rslc stack, dtype: `cupy.complexfloating`
                 is_shp:cp.ndarray, # shp bool, dtype: `cupy.bool`
                 block_size:int=128, # the CUDA block size, it only affects the calculation speed
                )-> tuple[cp.ndarray,cp.ndarray]: # the covariance and coherence matrix `cov` and `coh`
    '''
    Maximum likelihood covariance estimator.
    '''
    nlines, width, nimages = rslc.shape
    az_win, r_win = is_shp.shape[-2:]
    az_half_win = (az_win-1)//2
    r_half_win = (r_win-1)//2

    cov = cp.empty((nlines,width,nimages,nimages),dtype=rslc.dtype)
    coh = cp.empty((nlines,width,nimages,nimages),dtype=rslc.dtype)

    _emperical_co_kernel(rslc, is_shp, cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                    cp.int32(az_half_win),cp.int32(r_half_win),cov,coh,size = nlines*width,block_size=block_size)
    return cov,coh

The `cov` and `coh` is defined as:

$$
cov = E(z_1z_2^*) \quad coh=\frac{E(z_1z_2^*)}{\sqrt{E(|z_1|^2)E(|z_2|^2)}}
$$

and estimated as:

$$
cov = \frac{\sum_{i=1}^{L}z_1^{i}z_2^{i*}}{L} \quad coh = \frac{\sum_{i=1}^{L}z_1^{i}z_2^{i*}}{\sqrt(\sum_{i=1}^{L}|z_1^{i}|^2)(\sum_{i=1}^{L}|z_2^{i}|^2)}
$$

using all selected SHPs. Their shapes are [nlines,width,nimages,nimages].

The `rslc` is a three dimentional cupy `ndarray`. The `dtype` should be `cupy.complex64`. From outerest to innerest, the three dimentions are azimuth, range and image.
`is_shp` is a four dimentional cupy `ndarray`. It describes if pixels in the search window are SHP to the central pixel.
From outerest ot innerest, they are azimuth, range, secondary pixel relative azimuth, secondary pixel relative range.

Here is an example:

In [ ]:
rslc = zarr.open('../../data/rslc.zarr/','r')[600:605,600:605]
rslc = cp.asarray(rslc)
rslc.shape

(5, 5, 17)

`rslc` is a stack of 5 rslc images. Each of the image has 5 pixel in azimuth dimention and 10 pixels in range dimention.
Apply ks test on it:

In [ ]:
rmli = cp.abs(rslc)**2
sorted_rmli = cp.sort(rmli,axis=-1)
dist, p = ks_test(sorted_rmli,az_half_win=1,r_half_win=1)

Seclect SHP based on p value:

In [ ]:
is_shp = (p < 0.005) & (p >= 0.0)

Estimate the covarience and coherence matrix: 

In [ ]:
cov,coh = emperical_co(rslc,is_shp)
cov.shape, coh.shape

((5, 5, 17, 17), (5, 5, 17, 17))

Both `cov` and `coh` are complex data. The shape shows each covarience or coherence matrix is 5 by 5 since there are 5 images.
And `cov` and `coh` are matrix for all 5*10 pixels.

In [ ]:
#| hide
# test

# az, r, image, image
half_az_win = is_shp.shape[2]//2;
half_r_win = is_shp.shape[3]//2;
for i, j, k, l in itertools.product(range(rslc.shape[0]),range(rslc.shape[1]),range(rslc.shape[2]),range(rslc.shape[2])):
    _cov = 0.0+0.0j
    _amp2_k = 0.0
    _amp2_l = 0.0
    # shp_az, shp_r
    n_shp = 0
    for m, n in itertools.product(range(is_shp.shape[2]),range(is_shp.shape[3])):
        if is_shp[i,j,m,n]:
            _cov += rslc[i+m-half_az_win,j+n-half_r_win,k]*rslc[i+m-half_az_win,j+n-half_r_win,l].conj()
            _amp2_k += abs(rslc[i+m-half_az_win,j+n-half_r_win,k])**2
            _amp2_l += abs(rslc[i+m-half_az_win,j+n-half_r_win,l])**2
            n_shp+=1
    assert abs(_cov/n_shp-cov[i,j,k,l])<1.0e-6
    assert abs(_cov/math.sqrt(_amp2_k*_amp2_l) - coh[i,j,k,l]) < 1.0e-6

In [ ]:
#| export
# I is int32* or int64*
_emperical_co_sp_kernel = cp.ElementwiseKernel(
    'raw T rslc, raw I sp_idx, raw bool is_shp_sp, int32 nlines, int32 width, int32 nimages, int32 az_half_win, int32 r_half_win, int32 num_sp',
    'raw T cov, raw T coh',
    '''
    if (i >= num_sp) return;
    int az_win = 2*az_half_win+1;
    int r_win = 2*r_half_win+1;
    int win = az_win*r_win;
    
    int ref_az = sp_idx[i*2];
    int ref_r = sp_idx[i*2+1];

    int sec_az, sec_r;

    int m,j; // index of each coherence matrix
    int k,l; // index of search window
    T _cov; // covariance
    float _amp2_m; // sum of amplitude square for image i
    float _amp2_j; // sum of amplitude aquare for image j
    int rslc_inx_m, rslc_inx_j;
    int n; // number of shp

    for (m = 0; m < nimages; m++) {
        for (j = 0; j < nimages; j++) {
            _cov = T(0.0, 0.0);
            _amp2_m = 0.0;
            _amp2_j = 0.0;
            n = 0;
            for (k = 0; k < az_win; k++) {
                for (l = 0; l < r_win; l++) {
                    sec_az = ref_az-az_half_win+k;
                    sec_r = ref_r-r_half_win+l;
                    if (is_shp_sp[i*win+k*r_win+l] && sec_az >= 0 && sec_az < nlines && sec_r >= 0 && sec_r < width) {
                        rslc_inx_m = (sec_az*width+sec_r)*nimages+m;
                        rslc_inx_j = (sec_az*width+sec_r)*nimages+j;
                        _amp2_m += norm(rslc[rslc_inx_m]);
                        _amp2_j += norm(rslc[rslc_inx_j]);
                        _cov += rslc[rslc_inx_m]*conj(rslc[rslc_inx_j]);
                        n += 1;
                        //if (i == 0 && m ==3 && j == 1) {
                        //    printf("%f",_cov.real());
                        //}
                    }
                }
            }
            cov[(i*nimages+m)*nimages+j] = _cov/(float)n;
            //if ( i == 0 && m==3 && j ==1 ) printf("%d",((i*nimages+m)*nimages+j));
            _amp2_m = sqrt(_amp2_m*_amp2_j);
            coh[(i*nimages+m)*nimages+j] = _cov/_amp2_m;
        }
    }
    ''',
    name = 'emperical_co_sp_kernel',reduce_dims = False,no_return=True
)

In [ ]:
#| export
def emperical_co_sp(rslc:cp.ndarray, # rslc stack, dtype: `cupy.complexfloating`
                    sp_idx:cp.ndarray, # index of sparse data, dtype: `cupy.int`, shape: [n_sp,2]
                    is_shp_sp:cp.ndarray, # shp bool, dtype: `cupy.bool`
                    block_size:int=128, # the CUDA block size, it only affects the calculation speed
                   )-> tuple[cp.ndarray,cp.ndarray]: # the covariance and coherence matrix `cov` and `coh`
    '''
    Maximum likelihood covariance estimator for sparse data.
    '''
    nlines, width, nimages = rslc.shape
    az_win, r_win = is_shp_sp.shape[-2:]
    az_half_win = (az_win-1)//2
    r_half_win = (r_win-1)//2
    num_sp = sp_idx.shape[0]
    
    cov = cp.empty((num_sp,nimages,nimages),dtype=rslc.dtype)
    coh = cp.empty((num_sp,nimages,nimages),dtype=rslc.dtype)

    _emperical_co_sp_kernel(rslc, sp_idx, is_shp_sp, cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                    cp.int32(az_half_win),cp.int32(r_half_win),cp.int32(num_sp),cov,coh,size = num_sp,block_size=block_size)
    return cov,coh

`emperical_co_sp` is the `emperical_co` on sparse data, e.g., DSs. `rslc` is same as `emperical_co`, `sp_idx` is the index with shape [number_of_point,2], `is_shp_sp` is similar to `is_shp` in `emperical_co` but it only contains information about the sparse data. It is a 3D array with shape [number_of_point,az_win,r_win].

Compared with `emperical_co`, `emperical_co_sp` only estimate coherence/covariance at specific position so the memory usage is much small.

Example:

In [ ]:
rslc = zarr.open('../../data/rslc.zarr/','r')[600:650,600:650]
rslc = cp.asarray(rslc)
rmli = cp.abs(rslc)**2
sorted_rmli = cp.sort(rmli,axis=-1)
dist, p = ks_test(sorted_rmli,az_half_win=10,r_half_win=10)
is_shp = (p < 0.005) & (p >= 0.0)
cov, coh = emperical_co(rslc,is_shp)

In [ ]:
shp_num = cp.count_nonzero(is_shp,axis=(-2,-1))
is_ds = shp_num >= 50
is_shp_ds = is_shp[is_ds]
ds_idx = cp.vstack(cp.where(is_ds)).T
ds_idx.shape

(297, 2)

In [ ]:
cov_ds, coh_ds = emperical_co_sp(rslc,ds_idx,is_shp_ds)
coh_ds.shape

(297, 17, 17)

In [ ]:
assert_array_almost_equal(cov[is_ds],cov_ds)
assert_array_almost_equal(coh[is_ds],coh_ds)

## Covariance and Coherence Matrix Regularizer

In [ ]:
#| export
def wherePD(co:cp.ndarray, # absolute value of complex coherence/covariance stack
         )-> cp.ndarray: # bool array indicating wheather coherence/covariance is positive define
    L = cp.linalg.cholesky(co)
    isPD = cp.isfinite(L).all(axis=(-2,-1))
    return isPD

In [ ]:
#| export
'''
    The method is presented in [1]. John D'Errico implented it in MATLAB [2] under BSD
    Licence and [3] implented it with Python/Numpy based on [2] also under BSD Licence.
    This is a cupy implentation with stack of matrix supported.

    [1] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    
    [2] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
    
    [3] https://gist.github.com/fasiha/fdb5cec2054e6f1c6ae35476045a0bbd
'''
def nearestPD(co:cp.ndarray, # stack of matrix with shape [...,N,N]
             )-> cp.ndarray: # nearest positive definite matrix of input, shape [...,N,N]
    """Find the nearest positive-definite matrix to input matrix."""

    B = (co + cp.swapaxes(co,-1,-2))/2
    s, V = cp.linalg.svd(co)[1:]
    I = cp.eye(co.shape[-1],dtype=co.dtype)
    S = s[...,None]*I
    del s

    H = cp.matmul(cp.swapaxes(V,-1,-2), cp.matmul(S, V))
    del S, V
    A2 = (B + H) / 2
    del B, H
    A3 = (A2 + cp.swapaxes(A2,-1,-2))/2
    del A2

    if wherePD(A3).all():
        return A3
    
    co_norm = cp.linalg.norm(co,axis=(-2,-1))
    spacing = cp.nextafter(co_norm,co_norm+1.0)-co_norm
    
    k = 0
    while True:
        isPD = wherePD(A3)
        isPD_all = isPD.all()
        if isPD_all or k>=100:
            break
        k+=1
        mineig = cp.amin(cp.linalg.eigvalsh(A3),axis=-1)
        assert cp.isfinite(mineig).all()
        A3 += (~isPD[...,None,None] * I) * (-mineig * k**2 + spacing)[...,None,None]
    #print(k)
    return A3

`nearest` means the Frobenius norm of the difference is minimized.

Example:

In [ ]:
coh = zarr.open('../../data/emperical_coherence.zarr','r')[:500,:500]
coh = cp.asarray(coh)
coh = abs(coh)
coh.shape

(500, 500, 17, 17)

In [ ]:
nearest_coh = nearestPD(coh)

In [ ]:
#| export
def regularize_spectral(coh:cp.ndarray, # stack of matrix with shape [...,N,N]
                        beta:Union[float, cp.ndarray], # the regularization parameter, a float number or cupy ndarray with shape [...]
                        )-> cp.ndarray: # regularized matrix, shape [...,N,N]
    '''
    Spectral regularizer for coherence matrix.
    '''
    I = cp.eye(coh.shape[-1],dtype=coh.dtype)
    beta = cp.asarray(beta)[...,None,None]

    regularized_coh = (1-beta)*coh + beta* I
    return regularized_coh

`regularize_spectral` can regularize the absolute value of coherence matrix for better phase linking.
It is first presented in [@zwiebackCheapValidRegularizers2022a].

Examples:

In [ ]:
coh = zarr.open('../../data/emperical_coherence.zarr','r')[:500,:500]
coh = cp.asarray(coh)
coh = abs(coh)
coh.shape

(500, 500, 17, 17)

In [ ]:
regularized_coh1 = regularize_spectral(coh,0.1)

More general, `bata` can be a `cp.ndarray`:

In [ ]:
beta = cp.ones(coh.shape[:-2])/10
regularized_coh2 = regularize_spectral(coh,beta)

In [ ]:
#| hide
assert_array_almost_equal(regularized_coh1,regularized_coh2)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()